### This notebook is designed to combine and cleaning power outage customer imapct data (Y) and weather related predictor data (X)

#### 1. Prepare Y data

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [2]:
# Read and combine customer impact data
impact_2022 = pd.read_csv('raw_data/eaglei_outages_with_events_2022.csv')
impact_2023 = pd.read_csv('raw_data/eaglei_outages_with_events_2023.csv')
impact = pd.concat([impact_2022, impact_2023], ignore_index=True)

# Convert datetime column
impact['start_time'] = pd.to_datetime(impact['start_time'])

print(f"Combined data shape: {impact.shape}")
impact.head()

Combined data shape: (172194, 14)


,event_id,state_event,Datetime Event Began,Datetime Restoration,Event Type,fips,state,county,start_time,duration,end_time,min_customers,max_customers,mean_customers
0,Alabama-0,Alabama,2022-01-03 01:00:00,2022-01-03 14:00:00,Severe Weather,1001,Alabama,Autauga,2022-01-03 04:00:00,5.75,2022-01-03 09:45:00,452,723,583.478261
1,Alabama-0,Alabama,2022-01-03 01:00:00,2022-01-03 14:00:00,Severe Weather,1003,Alabama,Baldwin,2022-01-03 13:45:00,3.00,2022-01-03 16:45:00,738,1404,1096.666667
2,Alabama-0,Alabama,2022-01-03 01:00:00,2022-01-03 14:00:00,Severe Weather,1005,Alabama,Barbour,2022-01-03 03:45:00,11.50,2022-01-03 15:15:00,208,979,570.826087
3,Alabama-0,Alabama,2022-01-03 01:00:00,2022-01-03 14:00:00,Severe Weather,1007,Alabama,Bibb,2022-01-03 03:00:00,2.50,2022-01-03 05:30:00,252,254,252.400000
4,Alabama-0,Alabama,2022-01-03 01:00:00,2022-01-03 14:00:00,Severe Weather,1009,Alabama,Blount,2022-01-03 03:45:00,2.00,2022-01-03 05:45:00,295,299,296.875000


In [3]:
# Filter for California severe weather (Dec 2022 - Mar 2023)
# We can expand by using other States' training data for accuracy improvement later
impact_filtered = impact[
    (impact['state_event'] == 'California') & 
    (impact['Event Type'] == 'Severe Weather') & 
    (impact['start_time'] >= '2022-12-01') & 
    (impact['start_time'] <= '2023-03-31')
]
impact_filtered["start_date"] = impact_filtered['start_time'].dt.floor('D')
#impact_filtered = impact_filtered[["start_date", "county", "duration", "mean_customers"]]
impact_filtered = impact_filtered[["start_date", "county", "mean_customers"]]

/var/folders/yk/dpbmrr3541xcv9jkz422gf0r0000gn/T/ipykernel_20824/266098259.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impact_filtered["start_date"] = impact_filtered['start_time'].dt.floor('D')


In [4]:
impact_filtered.head(5)

,start_date,county,mean_customers
6806,2022-12-31,Amador,2693.000000
6807,2022-12-31,El Dorado,1321.333333
6808,2022-12-31,Napa,1205.000000
6809,2022-12-31,Sacramento,207.250000
6810,2022-12-31,San Luis Obispo,512.666667


In [5]:
impact_filtered.shape

(381, 3)

In [6]:
# Agg to county-date level
impact_agg = impact_filtered.groupby(["start_date", "county"]).sum().reset_index()
impact_agg.rename(columns={"mean_customers": "total_customers", "start_date": "date"}, inplace=True)
impact_agg.head(20)

,date,county,total_customers
0,2022-12-31,Amador,2693.000000
1,2022-12-31,El Dorado,1321.333333
2,2022-12-31,Napa,1205.000000
3,2022-12-31,Sacramento,207.250000
4,2022-12-31,San Luis Obispo,512.666667
5,2022-12-31,Solano,365.625000
6,2022-12-31,Sonoma,585.666667
7,2023-01-07,Monterey,1660.985294
8,2023-01-07,San Joaquin,948.500000
9,2023-01-07,Tehama,392.583333


In [7]:
impact_agg.shape

(224, 3)

#### 2. Preapre X data

In [8]:
predictor = pd.read_csv('raw_data/california_risk_model_data_dec2022_mar2023.csv')
predictor.rename(columns = {"county_name": "county"}, inplace = True)
predictor["date"] = pd.to_datetime(predictor["date"])
predictor["date"] = predictor['date'].dt.floor('D')

In [9]:
predictor.shape

(7018, 36)

In [10]:
predictor.head(5)

,county_code,county,date,ivt_max,ar_intensity,gage_height_mean,streamflow_mean,wind_speed_max,wind_gust_max,precip_total,runoff_total,temp_anomaly,soil_moisture_mean,IVT_max,IVT_duration,AR_category,Precip_24h,Precip_72h,Wind_gust_max,Soil_moisture_pct,API_7d,API_14d,Snowpack_SWE,Temp_anomaly,Streamflow_pct,Streamflow_p95_exceed,Runoff_ratio,Flood_stage_exceed,AR_count_7d,AR_count_14d,Wet_days_10,Dry_gap,Population_exposed,Infrastructure_density,County_area,Agricultural_share
0,6001,Alameda,2022-12-01,517.876160,AR2,3.363378,8.844656,10.958761,28.427376,3.846169,0.067472,2.004796,0.180444,517.876160,12,AR2,3.846169,3.846169,28.427376,0.180444,3.846169,3.846169,80,2.004796,56.250,-0.990279,0.017543,0,1.0,1.0,1.0,0,100000,2.0,5000,0.45
1,6001,Alameda,2022-12-02,231.908783,NaN,3.363423,9.136874,6.120897,11.851345,0.020504,0.000238,-2.666713,0.343442,231.908783,0,NaN,0.020504,3.866673,11.851345,0.343442,3.482056,3.674364,80,-2.666713,59.375,-0.989958,0.011628,0,1.0,1.0,1.0,1,100000,2.0,5000,0.45
2,6001,Alameda,2022-12-03,456.337891,AR1,3.365482,7.298010,5.717101,9.304350,0.570774,0.009298,-2.150843,0.315793,456.337891,12,AR1,0.570774,4.437447,9.304350,0.315793,3.666162,4.051805,80,-2.150843,46.875,-0.991979,0.016291,0,2.0,2.0,1.0,2,100000,2.0,5000,0.45
3,6001,Alameda,2022-12-04,550.994995,AR2,3.800255,41.210267,6.267160,22.702106,1.450539,0.032067,0.536351,0.388857,550.994995,12,AR2,1.450539,2.041817,22.702106,0.388857,4.672956,5.280471,80,0.536351,78.125,-0.954708,0.022107,0,3.0,3.0,2.0,0,100000,2.0,5000,0.45
4,6001,Alameda,2022-12-05,330.216705,AR1,3.439359,8.097739,6.848634,18.281908,0.118732,0.001907,3.223302,0.352034,330.216705,12,AR1,0.118732,2.140045,18.281908,0.352034,4.202890,5.104804,80,3.223302,53.125,-0.991100,0.016064,0,4.0,4.0,2.0,1,100000,2.0,5000,0.45


In [11]:
primary_table = impact_agg.merge(
    predictor, 
    left_on=['county', 'date'], 
    right_on=['county', 'date'],
    how='left'
)

print(f"Merged data shape: {primary_table.shape}")
primary_table.head()

Merged data shape: (224, 37)


,date,county,total_customers,county_code,ivt_max,ar_intensity,gage_height_mean,streamflow_mean,wind_speed_max,wind_gust_max,precip_total,runoff_total,temp_anomaly,soil_moisture_mean,IVT_max,IVT_duration,AR_category,Precip_24h,Precip_72h,Wind_gust_max,Soil_moisture_pct,API_7d,API_14d,Snowpack_SWE,Temp_anomaly,Streamflow_pct,Streamflow_p95_exceed,Runoff_ratio,Flood_stage_exceed,AR_count_7d,AR_count_14d,Wet_days_10,Dry_gap,Population_exposed,Infrastructure_density,County_area,Agricultural_share
0,2022-12-31,Amador,2693.000000,6005,781.385925,AR3,NaN,NaN,16.213619,36.041809,17.823219,4.023075,5.834319,0.476011,781.385925,12,AR3,17.823219,24.079323,36.041809,0.476011,29.768515,32.117653,20,5.834319,NaN,0.000000,0.225721,0,4.0,4.0,4.0,0,30000,2.0,3000,0.35
1,2022-12-31,El Dorado,1321.333333,6017,612.015625,AR2,6.725536,501.512348,16.793095,46.787708,24.123669,5.789280,5.107599,0.514730,612.015625,12,AR2,24.123669,31.211853,46.787708,0.514730,38.038540,40.787506,80,5.107599,96.875,0.940605,0.239983,1,4.0,4.0,4.0,0,30000,2.0,3000,0.35
2,2022-12-31,Napa,1205.000000,6055,1174.133789,AR4,12.444826,1326.107639,12.255922,37.610329,3.623009,0.169039,5.680211,0.410417,1174.133789,12,AR4,3.623009,5.913258,37.610329,0.410417,8.080816,9.077239,80,5.680211,96.875,0.459083,0.046657,1,4.0,4.0,3.0,0,30000,2.0,3000,0.35
3,2022-12-31,Sacramento,207.250000,6067,1035.189819,AR4,15.933702,11063.678186,15.634851,28.197020,14.045238,3.161669,6.739041,0.437250,1035.189819,12,AR4,14.045238,18.409252,28.197020,0.437250,23.039103,24.982715,80,6.739041,96.875,0.350062,0.225106,1,3.0,3.0,3.0,0,30000,2.0,3000,0.35
4,2022-12-31,San Luis Obispo,512.666667,6079,752.332153,AR3,7.198949,72.212894,13.706140,33.783848,0.730991,0.014067,4.653655,0.404371,752.332153,12,AR3,0.730991,0.884533,33.783848,0.404371,2.598190,3.055525,20,4.653655,84.375,-0.889952,0.019243,0,5.0,5.0,2.0,3,240000,5.0,8000,0.15


In [12]:
primary_table.to_csv("cleaned_primary_table/conditional_impact_primary_table.csv", index=False)